# Herramientas para hacer Fast Fourier Transform Directo

En este cuaderno de Julia indicamos un posible uso de las herramientas desarrolladas por el Dr. WPK Zapfe para la transformada rápida de Fourier en linea de los datos de BioCAM X. Estamos usando ad hoc un par de modulos auxiliares que nos permiten abrir los archivos tal y como vienen en el flujo (hexadecimal sin signo), interpretarlos y aplicarles una fftw.

In [7]:
push!(LOAD_PATH, ".") # las herramientas para hacer fftw estan aqui
push!(LOAD_PATH, "../Preprocesamiento/") # el interpretador de los datos crudos esta aqui
using PreprocTools #para abrir y manejar los datos crudos
using FFTWtoolkit, FFTW # the fastest fourier transform in the west
using PyPlot # posible graficador, se pueden usar otros

In [8]:
abrestring="/home/karel/BRWFiles/Facilitada/19115s1cut_single_event.brw"


"/home/karel/BRWFiles/Facilitada/19115s1cut_single_event.brw"

En la versión en linea, los datos que aparecen catalogados en el siguiente diccionario deben de estar detallados en algun objeto visible de python. De aquí los interesantes son minvolt, maxvolt, bithdept e inversión, de los que se obtiene el factor de conversión según la siguiente formula
$$
(maxvolt-minvolt)/2^(bitdepth)*inversion=factor
$$
Y también nos interesa la frecuencia de muestreo. El número de cuadros máximo no tiene sentido en el trabajo en linea, ese se va a obtener por la convenciencia del usuario, pero para una buena fftw unos 1000 cuadros son adecuados.

In [9]:
Datos=AbreyCheca(abrestring)

Dict{String,Any} with 8 entries:
  "minvolt"     => -4125.0
  "maxvolt"     => 4125.0
  "frecuencia"  => 7022.06
  "DatosCrudos" => UInt16[0x0fff 0x0fff … 0x0fff 0x0fff; 0x0814 0x07ed … 0x0800…
  "bitdepth"    => 0x0c
  "factor"      => 2.01416
  "numcuadros"  => 2000
  "duracion"    => 0.284817

Las siguientes constantes nos daran las unidades adecuadas para trabajar.

In [10]:
freq=Datos["frecuencia"]
factor=Datos["factor"]

2.01416015625

Esta es una función de conveniencia para escoger el canal. En la versión en linea usar el ratón de la computadora sobre la representación visual del MEA puede ser más conveniente.

In [11]:
escogecanal(y::Int,x::Int)=(y-1)*64+x

escogecanal (generic function with 1 method)

Se le preguntará al usuario cuanto en tiempo quiere transformar a la Fourier. El usuario especificará un rango de tiempo (típicamente entre 0.1 y 1 segundo), y eso se transformará en cuadros.

In [12]:
intertiempo=0.2848

0.2848

In [13]:
timetoframe(t)=round(Int, floor(t*freq))

timetoframe (generic function with 1 method)

In [14]:
inter1=timetoframe(intertiempo) #un putativo numero de cuadros que tengamos para hacer fftw

1999

In [15]:
electo=escogecanal(44,26) # un canal que tal vez se puede escoger con el ratón (de la compu, no del experimento)

2778

Aquí estoy usando el máximo número de cuadros que hay en el archivo como límite para el rango de la fftw. En la versión en linea debe ser posible extenderlo mucho más, simplemente esperando unos cuantos segundos de muestreo y ya.

In [16]:
if (inter1 < Datos["numcuadros"] )
        nmax=inter1
else
        nmax= Datos["numcuadros"]
end

1999

Tomamos el arreglo de los datos crudos escogidos (viene en UInt16, enteros sin signo de 16 bytes)

In [17]:
lfprawsel=Datos["DatosCrudos"][electo,:];
typeof(lfprawsel)

Array{UInt16,1}

A continuación hacemos variables auxiliares, para que la representación se encuentre en las unidades adecuadas.Por ejemplo, nos gustaría que la transformada de Fourier tuviera una escala en Hz positivos.

In [18]:
rango=1:nmax
tmax=nmax/freq
rangotiempos=(rango.-1)./freq
rangofreqs=(rango.-1)./tmax
# la fftw es simetrica con el tiempo, aqui corregimos el efecto
rfreqsim=map(x->-freq/2+(x-1)*freq/(nmax-1),rango);  
# establecemos las medidas del grafico (estas seguramente se pueden obtener del feed del aparato)
vunits="mV"
tunits="s"
funits="Hz"
punits=vunits*"^2·"*tunits

"mV^2·s"

Hacemos la conversión a las unidades SI.

In [19]:
lfprawsel=Datos["DatosCrudos"][electo,(end-nmax)+1:end];# si el nmax NO concide con ncuadros, es decir, es menor
lfprawsel=(lfprawsel.-2048).*factor; # los datos en mV

El 3Brain ya grafica los voltajes, esto es solo una ayuda para mi,

In [20]:
plot(rangotiempos,lfprawsel)

ArgumentError: ArgumentError: hasproperty of NULL PyObject

In [21]:
1/tmax # la minima frecuencia observable.

3.5127854920187214

Por cada tipo de array, solo es necesario calcular el plan una vez. Asi que si el usuario cambia de electrodo, no es necesario volver a calcular el plan, pero si cambia la longitud entonces si.

In [ ]:
plan=plan_fft(lfprawsel,  flags=FFTW.MEASURE)

Aplicamos la transformada númerica de Fourier ahora si. El plan se aplica con " * " en julia. Lo simetrizamos, y calculamos el poder (el valor absoluto al cuadrado)

In [ ]:
fttotal=(plan*lfprawsel);
ftsym=circshift(fttotal, div(nmax,2));
epoder=map(abs2, ftsym);

Finalmente la prueba: una hermosa gráfica del espectro de poder por frecuencia.

In [ ]:
figura=figure(figsize=(12,6))
grid(which="both")
fminplot=(1/tmax*0.95)
fmaxplot=freq/2*1.05
xlim(fminplot,fmaxplot)
xlabel(funits)
ylabel(punits)
#yscale("log")
#xscale("log")
#yscale("log")
mediorango=1:500
plot(rfreqsim,epoder, lw=0.74)
#outname=palabra*"-sweep_$nsweep.png"
#savefig(outname,dpi=90)